In [ ]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Introduction

#### Aim

* Extract the HQ curves of the Heart Mate 3 device, for different RPMs. We constraint the curves to be polynomials of degree two.
* The coefficients of the curves will be reported in the `VAD2` model in the `Mathcard.mo` file.

#### Original HQ curve

In [ ]:
from IPython.display import Image
Image("../res/modelica/HQ-curve-HMIII.png")

# Curve coefficients

#### Report data points


In [ ]:
# Dictionnary: RPM [int] -> data [list of xs [list] and ys[list] ]
# x = Q, y = P
data = {}

In [ ]:
data[3000] = [
    [0.0, 1.0, 2.0, 2.9], # flow
    [32,  29, 23, 14],   # pressure
]

In [ ]:
data[4000] = [
    [0.0, 2.0, 3.0, 4.5],
    [58,  49,  41,  16]
]

In [ ]:
data[5000] = [
    [0.0, 2.0, 4.0, 6.0],
    [90,  80,  63,  19]
]

In [ ]:
data[6000] = [
    [0.0, 2.0, 4.0, 6.0, 7.0, 7.4],
    [131, 120, 105, 70,  36,  22]
]

In [ ]:
data[7000] = [
    [0.0, 2.0, 4.0, 5.0, 8.0],
    [177, 168, 150, 140, 63]
]

In [ ]:
data[8000] = [
    [0.0, 2.0, 3.0, 5.0, 6.0, 8.0, 10.0],
    [232, 220, 211, 191, 178, 126, 43]
]

In [ ]:
data[9000] = [
    [0.0, 2.0, 4.0, 6.0, 9.0, 10.0, 11.0],
    [290, 278, 263, 240, 164, 125,  72]
]

In [ ]:
pickle.dump(data, open('../res/modelica/HM3-HQ-curve-data.bin', 'wb'))

#### Adjust units

The reported flow rate values are in $L/min$, while the model assumes $ml/s$. Thus we have to multiply every measure of $Q$ by $1000/60$.

In [ ]:
for RPM in data:
    data[RPM] = np.array(data[RPM])
    data[RPM][0] = data[RPM][0] * 1000 / 60

## Fit polynomials


#### Procedure 

1. We reconstruct the HQ curve with the same axes as the original plot, as a mean of comparison
2. We reconstruct the HQ curve with **inverted axes**,
    * i.e. we perform polynomial regression with P as the input and Q as the output

#### Demo

In [ ]:
# Demo numpy.polyfit: f(x)=x^2
x = [-1,0,1, 1.2]
y = [1,0,1, 2]
coefs = np.polyfit(x, y, 2)
coefs # order: x^n, x^n-1, ..., x^1, x^0
print(coefs)
plt.subplots(1,1,figsize=(6,3))
plt.scatter(x, y)
x = np.linspace(-2, 2, 100)
plt.plot(x, np.polyval(coefs, x));

#### Compute coefficients of the polynomials

In [ ]:
# Pressure in function of flow, dP(Q)
coefs = {}
# Flow in function of pressure, Q(dP)
coefs_rev = {}

for k in data:
    x = data[k][0]
    y = data[k][1]
    coefs[k] = np.polyfit(x, y, deg=2)
    coefs_rev[k] = np.polyfit(y, x, deg=2)

In [ ]:
coefs

In [ ]:
coefs_rev

#### HQ plots

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,5))
Qmax = 12 * 1000/60
Q = np.linspace(0, Qmax, 1000)

for RPM, coef in coefs.items():
    plt.scatter(data[RPM][0], data[RPM][1])
    plt.plot(Q, np.polyval(coef, Q))

plt.xlim(0, Qmax); plt.ylim(0, 300); plt.grid(ls='--');
plt.xlabel("Flow Q [ml/s]"); plt.ylabel("Pressure head dP [mmHg]");
plt.title("Reconstructed HQ curve");
plt.savefig('../res/modelica/HM3-HQ-curve-reconstructed.svg')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5, 10))
for RPM, coef in coefs_rev.items():
    pmin = min(data[RPM][1])
    pmax = max(data[RPM][1])
    dP = np.linspace(pmin, pmax, 1000)
    Q = np.polyval(coef, dP)
    plt.scatter(data[RPM][1], data[RPM][0])
    plt.plot(dP, Q, label=RPM)
    
plt.xlabel("Pressure head dP [mmHg]"); plt.ylabel("Flow Q [ml/s]"); plt.legend();
plt.grid(ls='--'); plt.title("Q as function of pressure head dP");

# Compare with the HQ curve of HM2

## Coefficients

Code provided from the `VAD` model in modelica:

```
Q = if RPM == 10000 then min(306.65 - 4.03 * dP + 0.0127 * dP ^ 2, 133) else if RPM == 9000 then min(245.16 - 3.6 * dP + 0.0119 * dP ^ 2, 116.667) else if RPM == 8000 then min(195.85 - 3.506 * dP + 0.0136 * dP ^ 2, 100) else 0;```


In [ ]:
coef = {}
mins = {}

In [ ]:
coef[10000] = [0.0127, -4.03, 306.65]
coef[9000]  = [0.0119, -3.6,  245.16]
coef[8000]  = [0.0136, -3.506, 195.85]

In [ ]:
mins[10000] = 133
mins[9000] = 116.667
mins[8000] = 100

## Plotting without `min` applied

In [ ]:
dP = np.linspace(0, 300, 1000)
plt.subplots(1,1,figsize=(6,4))

for elem in coef :
    Q = np.polyval(coef[elem], dP)
    plt.plot(dP, Q, label=elem)

plt.grid(ls='--'); plt.xlabel("dP"); plt.ylabel("Q");
plt.legend();

## Apply `min`

In [ ]:
curves = {}
dP = np.linspace(0, 300, 1000)
for elem in coef :
    Q = np.polyval(coef[elem], dP)
    curves[elem] = Q

In [ ]:
# Apply the min
curves_corrected = {}
for RPM in curves : 
    curves_corrected[RPM] = [ min(mins[RPM], q) for q in curves[RPM] ]

In [ ]:
plt.subplots(1,1,figsize=(6,4))
dP = np.linspace(0, 300, 1000)
for RPM in curves_corrected:
    plt.plot(dP, curves_corrected[RPM], label=RPM)
    
plt.grid(ls='--'); plt.xlabel("dP"); plt.ylabel("Q");
plt.legend();

# Reconsider the range of values for `dP`

In [ ]:
Image("../res/modelica/LVAD_dP.png")

The blue curve `LVAD2.dP` on top is for HM3, the red curve `LVAD.dP` is for HM2.

## HM2 HQ curve restricted in the observed dP range

In [ ]:
dPmax = 85
plt.subplots(1,1,figsize=(6,4))
for RPM in curves_corrected : 
    plt.plot(dP, curves_corrected[RPM], label=RPM)
    
plt.grid(ls='--'); plt.xlabel("dP"); plt.ylabel("Q");
plt.legend(); plt.xlim(0, dPmax);
plt.title("HMII HQ curve restricted in the observed dP range");

## HM3 HQ curve: Determine points where Q start to decrease

Given $Q = a_2 x^2 + a_1 x + a_0$, with x=dP, we calculate points where $Q'(x) = 0$, i.e. we calculate the x such that $x = - \frac{a_1}{2a_2}$

In [ ]:
dP_critical = {}

for RPM in coefs_rev : 
    dP_critical[RPM] = - coefs_rev[RPM][1] / (2 * coefs_rev[RPM][0])
print(dP_critical)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,5))

dPmax_HMIII = 300
dP = np.linspace(0, dPmax_HMIII, 1000)


for RPM in dP_critical : 
    # Compute Q
    Q = np.polyval(coefs_rev[RPM], dP)
    # Compute the points where Q begins to decrease
    a0, a1, a2 = coefs_rev[RPM][2], coefs_rev[RPM][1], coefs_rev[RPM][0]
    crit = dP_critical[RPM]
    q_crit = a0 + a1 * crit + a2 * crit ** 2
    # Once Q starts to decrease, set a constant
    Q = [ q if dP[i] >= crit else q_crit for i,q in enumerate(Q) ]
    
    print("RPM={}, (dP_crit, Q_crit) = ({:.5}, {:.5})".format(RPM, crit, q_crit))
    # Plot the cross marker
    plt.scatter(crit, q_crit, marker='x', color="red")
    ## Plot the curve
    plt.plot(dP, Q, label=RPM)
    
plt.xlim(0, dPmax_HMIII); plt.ylim(0, 200); plt.legend();
plt.xlabel("dP [mmHg]"); plt.ylabel("Q [mL/s]");
plt.title('$Q$ in function of $dP$  as implemented in Modelica')
plt.savefig('../res/modelica/HM3-HQ-curve-reversed.jpg')

# Article figure

In [ ]:
from ipypublish import nb_setup
import seaborn as sns

In [ ]:
plt = nb_setup.setup_matplotlib(usetex=False)
sns.set(style='whitegrid')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))

dPmax_HMIII = 300
dP = np.linspace(0, dPmax_HMIII, 1000)

for RPM in dP_critical : 
    # Compute Q
    Q = np.polyval(coefs_rev[RPM], dP)
    # Compute the points where Q begins to decrease
    a0, a1, a2 = coefs_rev[RPM][2], coefs_rev[RPM][1], coefs_rev[RPM][0]
    crit = dP_critical[RPM]
    q_crit = a0 + a1 * crit + a2 * crit ** 2
    
    # Once Q starts to decrease, set a constant
    Q = [ q if dP[i] >= crit else q_crit for i,q in enumerate(Q) ]
    
    print("RPM={}, (dP_crit, Q_crit) = ({:.5}, {:.5})".format(RPM, crit, q_crit))
    # Plot the cross marker
    plt.scatter(crit, q_crit, marker='x', color="black", zorder=20)
    ## Plot the curve
    plt.plot(dP, Q, label=RPM)
    
plt.xlim(0, dPmax_HMIII); plt.ylim(0, 200); plt.legend();
plt.xlabel("$\Delta$P [mmHg]"); plt.ylabel("Q [mL/s]");
plt.legend(title='RPM')
plt.grid(ls='--')
#plt.title('$Q$ in function of $dP$  as implemented in Modelica')
#plt.savefig('../res/modelica/HM3-HQ-curve-reversed.jpg')
plt.tight_layout()
plt.savefig('figs/LVAD-HQ-curve.eps')
plt.savefig('figs/LVAD-HQ-curve.pdf')

# Interpolate data at intermediate pump speed

## Reverse ramp test

The reverse ramp test consists of a progressive decrease in pump speed, by steps of 100 RPM. We want to calibrate DNNs for pump configurations between 6000 and 4000 RPMs (i.e. 21 configurations), and thus we need to interpolate curves at those intermediate speeds. 

In [ ]:
# Rationale: we can simply interpolate the polynomial coefficients
plt.figure(figsize=(6, 2.5))
p1 = [1.5, 0.5, 1]
p2 = [1, 1, 0]
x = np.linspace(-2, 2, 1000)
y1 = np.polyval(p1, x)
y2 = np.polyval(p2, x)
y = np.polyval([(e1+e2)/2 for e1,e2 in zip(p1, p2)], x)
plt.plot(x, y1, x, y2, x, y);

In [ ]:
def interp_curve(coefs_dict, target_speed):
    """Returns the coefficients of the curve corresponding to target_speed, 
    obtained by linear interpolation between closest available data."""
    if target_speed in coefs_dict:
        return list(coefs_dict[target_speed])
    
    speeds = sorted(list(coefs_dict))
    if target_speed < speeds[0] or target_speed > speeds[-1]:
        print(f'Warning: Target speed {target_speed} out of bounds.')
        if target_speed < speeds[0]:
            lower, upper = speeds[:2]
        else:
            lower, upper = speeds[-2:]
    else:
        # Find closest RPM levels on which to perform interpolation
        lower = max(filter(lambda x: x < target_speed, speeds))
        upper = min(filter(lambda x: x > target_speed, speeds))
    
    relationships = [
        np.polyfit([lower, upper], [c1, c2], deg=1)
        for c1, c2 in zip(coefs_dict[lower], coefs_dict[upper])
    ]
    
    coeffs = [
        np.polyval(p, target_speed)
        for p in relationships
    ]
    
    return coeffs

In [ ]:
interp_curve(coefs_rev, 3900)

In [ ]:
curves = {
    speed: interp_curve(coefs_rev, speed)
    for speed in range(4000, 6100, 100)
}
curves

In [ ]:
def compute_critical_values(coefs):
    xcrit = - coefs[1] / (2*coefs[0])
    ycrit = np.polyval(coefs, xcrit)
    return xcrit, ycrit

In [ ]:
def plot_interp_curves(coefs_dict, speeds, dp_max=100, q_max=200):
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    
    dp = np.linspace(0, dp_max, 1000)
    
    for s in speeds:
        coefs = interp_curve(coefs_dict, s)
        #dp_crit = - coefs[1] / (2*coefs[0])
        #q_crit = np.polyval(coefs, dp_crit)
        dp_crit, q_crit = compute_critical_values(coefs)
        q = np.polyval(coefs, dp)
        q = [
            q_ if dp[i] >= dp_crit else q_crit
            for i, q_ in enumerate(q)
        ]
        pltparams = ('-', dp_crit, q_crit, 'xr') \
                    if s in coefs_dict else ('k--', )
        if s in coefs_dict:
            ax.plot(dp, q, label=str(s))
            ax.plot(dp_crit, q_crit, 'xr')
        else:
            ax.plot(dp, q, 'k', lw=.5)
        #ax.plot(dp_crit, q_crit, 'o')
    
    ax.set_xlabel('Pump differential pressure [mmHg]')
    ax.set_ylabel('Pump blood flow [mL/s]')
    ax.axis(ymin=0, ymax=q_max)
    ax.legend(title='RPM')

In [ ]:
plot_interp_curves(coefs_rev, range(2000, 8100, 100), dp_max=250, q_max=175)

## Format to modelica code

The HQ curves are hard-coded in Modelica, we provide here an automatic output from the previously computed coefficients of the curves. Here is an example of what the Modelica code looks like:

```
  Q = max(0, 
    if RPM == 3000 then 
      if dP < 12.33 then 48.43 
      else (-0.12205 * dP ^ 2) + 3.0093 * dP + 29.886 
    else if RPM == 4000 then 
      if dP < 18.94 then 75.27 
      else (-0.048803 * dP ^ 2) + 1.8484 * dP + 57.770 
    else if RPM == 5000 then 
      if dP < 26.36 then 101.17 
      else (-0.024594 * dP ^ 2) + 1.29678 * dP + 84.081 
    else if RPM == 6000 then 
      if dP < 33.71 then 120.83 
      else (-0.012153 * dP ^ 2) + 0.81934 * dP + 107.02 
    else if RPM == 7000 then 
      if dP < 78.76 then 136.57 
      else (-0.013720 * dP ^ 2) + 2.1611 * dP + 51.474 
    else if RPM == 8000 then 
      if dP < 63.73 then 166.28 
      else (-0.0055313 * dP ^ 2) + 0.70504 * dP + 143.71 
    else if dP < 93.82 then 178.95 
    else (-0.0042916 * dP ^ 2) + 0.80525 * dP + 141.18
  );
```



* there is an `if` for each RPM level
    * for each RPM level, we test whether `dP` is smaller than the critical value

In [ ]:
def format_to_modelica(coefs_dict, indent='  ', error_value='10000'):
    def format_level(speed, coefs, indent):
        x_crit, y_crit = compute_critical_values(coefs)
        t = f'if RPM == {speed} then\n{indent}'
        t += 'if dP < {:.4} then {:.4}\n{}'.format(x_crit, y_crit, indent)
        t += 'else ( {:.6} * dP ^ 2 ) + {:.6} * dP + {:.6}'.format(*coefs)
        return t
    
    speeds = sorted(list(coefs_dict))
    # Special case: 1st if
    ifelse = format_level(speeds[0], coefs_dict[speeds[0]], indent)
    # Else if
    for s in speeds[1:]:
        ifelse += '\n' + 'else ' + format_level(s, coefs_dict[s], indent)
    # Last else: non-existing RPM value
    ifelse += '\nelse ' + error_value
    
    # Reindent
    ifelse = '\n'.join(indent*2 + line for line in ifelse.split('\n'))
    
    # Wrap around max function
    ifelse = indent + 'Q = max(0,\n' + ifelse + '\n' + indent + ');'
    
    return ifelse

In [ ]:
to_modelica = {
    speed: interp_curve(coefs_rev, speed)
    for speed in range(2000, 8100, 100)
}
to_modelica.update(coefs_rev)
print(list(to_modelica))

In [ ]:
print(format_to_modelica(to_modelica))